In [8]:
import pandas as pd
import os
from collections import Counter
import itertools
from tqdm import tqdm_notebook as tqdm
from operator import itemgetter

import sys
sys.path.append('/usr0/home/mamille2/textClassifier')
from textClassifierHATT import DataHandler

# data_dirpath = '/usr2/mamille2/tumblr/data' # erebor
data_dirpath = '/usr0/home/mamille2/tumblr/data' # misty

Using Theano backend.
Using cuDNN version 7005 on context None
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:04:00.0)


# Find mutual info between tags and identity categories

In [6]:
# Load data
posts = pd.read_pickle(os.path.join(data_dirpath, 'textposts_100posts.pkl'))
print(posts.shape)
posts.columns

(2026600, 29)


Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'body_toks', 'body_str', 'body_toks_no_titles',
       'body_toks_str_no_titles', 'parsed_tags_minfreq3',
       'parsed_tags_minfreq3_str'],
      dtype='object')

In [22]:
descs = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100_100posts.pkl'))
print(len(descs))
descs.columns

20266


Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description', 'age',
       'gender', 'sexual orientation', 'pronouns', 'personality type',
       'ethnicity/nationality', 'relationship status', 'age_terms',
       'gender_terms', 'sexual orientation_terms', 'pronouns_terms',
       'personality type_terms', 'ethnicity/nationality_terms',
       'relationship status_terms', 'location_terms', 'location',
       'roleplay_terms', 'roleplay', 'fandoms_terms', 'fandoms',
       'interests_terms', 'interests', 'weight_terms', 'weight',
       'gender/sexuality_terms', 'gender/sexuality', 'roleplay/fandoms_terms',
       'roleplay/fandoms'],
      dtype='object')

In [ ]:
# Get training set IDs

# Load DataHandler
dh = DataHandler(data_dirpath)
dh.load_processed_data('textposts_100posts_tags_posts')

In [17]:
all_ids = dh.tids
len(all_ids)

20266

In [18]:
train_ids = [tid for tid in all_ids if not tid in dh.tids_split['text'] and not tid in dh.tids_split['dev']]
len(train_ids)

16214

In [20]:
train_posts = posts[posts['tumblog_id'].isin(train_ids)]
len(train_posts)

1621400

In [23]:
train_descs = descs[descs['tumblog_id'].isin(train_ids)]
len(train_descs)

16214

In [25]:
# Build counters of tags with categories, as well as tags and categories by themselves
with open(os.path.join(data_dirpath, 'identity_categories.txt')) as f:
    categories = f.read().splitlines()
print(len(categories))
    
tags = set([t for l in train_posts['parsed_tags_minfreq3'].tolist() for t in l])
print(len(tags))

14
88877


In [48]:
# Drop duplicate posts and descriptions
train_posts.drop_duplicates('post_id', inplace=True)
print(len(train_posts))

train_descs.drop_duplicates('tumblog_id', inplace=True)
print(len(train_descs))

/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


16214

In [26]:
category_ctr = {cat: sum(train_descs[cat]) for cat in categories}
category_ctr

{'age': 2857,
 'gender': 1590,
 'pronouns': 1985,
 'sexual orientation': 1043,
 'weight': 366,
 'relationship status': 355,
 'ethnicity/nationality': 584,
 'location': 1084,
 'interests': 2655,
 'fandoms': 1689,
 'roleplay': 1626,
 'personality type': 732,
 'gender/sexuality': 3758,
 'roleplay/fandoms': 2794}

In [30]:
tag_ctr = Counter([t for l in train_posts['parsed_tags_minfreq3'].tolist() for t in l])
tag_ctr.most_common(20)

[('text', 8050),
 ('me', 6464),
 ('ifttt', 6037),
 ('ooc', 5897),
 ('appless_rp', 5667),
 ('personal', 5330),
 ('q', 4962),
 ('txt', 4788),
 ('nsfw', 4563),
 ('oc_rp', 4471),
 ('rp', 3289),
 ('c', 3289),
 ('twitter', 3109),
 ('new_rp', 3093),
 ('chat', 2994),
 ('ic', 2920),
 ('queue', 2785),
 ('convo', 2674),
 ('town_rp', 2023),
 ('reblog', 1819)]

In [28]:
# Examine certain tags
tag = 'town_rp'
tag_posts = train_posts[train_posts['parsed_tags_minfreq3'].map(lambda x: tag in x)]
print(len(tag_posts))
tag_posts.sample(100)

2157


,post_id,activity_time_epoch,tumblog_id,post_title,post_short_url,post_type,post_caption,post_format,post_note_count,created_time_epoch,...,root_post_id,body,mentions,post_tags,body_toks,body_str,body_toks_no_titles,body_toks_str_no_titles,parsed_tags_minfreq3,parsed_tags_minfreq3_str
811039,157032242604,1486675353000,76756998,NaN,https://tmblr.co/Zsa81u2IFsTsi,text,NaN,html,0,1486675353000,...,-1,<blockquote><p><small>I’ve got myself a bit of...,{},"{(appless rp),(rp),(town rp),(oc rp),(roleplay)}","[i, ’ve, got, myself, a, bit, of, a, headache,...",i ’ve got myself a bit of a headache but it ’d...,"[i, ’ve, got, myself, a, bit, of, a, headache,...",i ’ve got myself a bit of a headache but it ’d...,"[appless_rp, rp, town_rp, oc_rp, roleplay]",appless_rp rp town_rp oc_rp roleplay
813431,157032642554,1486675986000,76756998,NaN,https://tmblr.co/Zsa81u2IFt-Vw,text,NaN,html,0,1486675986000,...,-1,<blockquote><p><small>We’re a subplot group se...,{},"{(appless rp),(rp),(town rp),(city rp),(oc rp)}","[we, ’re, a, subplot, group, set, in, sacramen...","we ’re a subplot group set in sacramento , cal...","[we, ’re, a, subplot, group, set, in, sacramen...","we ’re a subplot group set in sacramento , cal...","[appless_rp, rp, town_rp, city_rp, oc_rp]",appless_rp rp town_rp city_rp oc_rp
815957,157033079839,1486676685000,76756998,NaN,https://tmblr.co/Zsa81u2IFvgGV,text,NaN,html,0,1486676685000,...,-1,<blockquote><p><small>We briefly talked about ...,{},"{(appless rp),(rp),(town rp),(oc rp),(roleplay...","[we, briefly, talked, about, having, an, event...",we briefly talked about having an event someti...,"[we, briefly, talked, about, having, an, event...",we briefly talked about having an event someti...,"[appless_rp, rp, town_rp, oc_rp, roleplay, sub...",appless_rp rp town_rp oc_rp roleplay subplot_rp
1409581,157075866939,1486765088000,76756998,NaN,https://tmblr.co/Zsa81u2IISuKx,text,NaN,html,0,1.48677e+12,...,-1,<blockquote><p><small>Faces like Ariana Grande...,{},"{(appless rp),(rp),(town rp),(roleplay),(oc rp)}","[faces, like, ariana, grande, ,, bella, hadid,...","faces like ariana grande , bella hadid , tyler...","[faces, like, ariana, grande, ,, bella, hadid,...","faces like ariana grande , bella hadid , tyler...","[appless_rp, rp, town_rp, roleplay, oc_rp]",appless_rp rp town_rp roleplay oc_rp
1749569,157072028274,1486758565000,76756998,NaN,https://tmblr.co/Zsa81u2IIEF9o,text,NaN,html,0,1.48676e+12,...,-1,<blockquote><p><small>We just reopened a coupl...,{},"{(appless rp),(rp),(town rp),(oc rp),(roleplay)}","[we, just, reopened, a, couple, spots, which, ...",we just reopened a couple spots which means we...,"[we, just, reopened, a, couple, spots, which, ...",we just reopened a couple spots which means we...,"[appless_rp, rp, town_rp, oc_rp, roleplay]",appless_rp rp town_rp oc_rp roleplay
1753782,157072818309,1486759961000,76756998,NaN,https://tmblr.co/Zsa81u2IIHG25,text,NaN,html,0,1.48676e+12,...,-1,<blockquote><p><small>I’m here waiting for the...,{},"{(appless rp),(rp),(town rp),(oc rp),(city rp)}","[i, ’m, here, waiting, for, the, mail, to, com...",i ’m here waiting for the mail to come and del...,"[i, ’m, here, waiting, for, the, mail, to, com...",i ’m here waiting for the mail to come and del...,"[appless_rp, rp, town_rp, oc_rp, city_rp]",appless_rp rp town_rp oc_rp city_rp
1755739,157073162555,1486760534000,280802399,NaN,https://tmblr.co/ZTpmsh2IIIa4x,text,NaN,html,0,1.48676e+12,...,-1,<blockquote><p> <small><b><i>( queue ) </i></...,{},"{(rp),(appless rp),(oc rp),(apartment rp),(tow...","[(, queue, ), —, if, you, ’re, seeing, this, i...",( queue ) — if you ’re seeing this it means i ...,"[(, queue, ), —, if, you, ’re, seeing, this, i...",( queue ) — if you ’re seeing this it means i ...,"[rp, appless_rp, oc_rp, apartment_rp, town_rp,...",rp appless_rp oc_rp apartment_rp town_rp queue...
1761861,157074284074,1486762422000,76756998,NaN,https://tmblr.co/Zsa81u2IIMrug,text,NaN,html,0,1.48676e+12,...,-1,<blockquote><p><small>If yo

In [43]:
# Join with identity annotations by tumblr id
merged = pd.merge(train_posts, train_descs, how='left', on=['tumblog_id']).loc[:, ['tumblog_id', 'parsed_tags_minfreq3', 'parsed_blog_description'] + categories]
merged.shape

(1529423, 17)

In [49]:
# Build lists of aggregate tags for users
gped = merged.groupby('tumblog_id') # Could do this directly from posts
len(gped)

16214

In [52]:
sum([['a'], ['b']], [])

['a', 'b']

In [58]:
agg_tags = gped.agg({'parsed_tags_minfreq3': lambda x: sum(x, [])})
len(agg_tags)

16214

In [60]:
agg_tags

,parsed_tags_minfreq3
tumblog_id,
44928,"[ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, twitter, tweets, ifttt, ...]"
628265,"[queer_poetry, indigenous_poetry, spilled_ink, artists_on_tumblr, tumblr_poetry, queer_poetry, indigenous_poetry, spilled_ink, spilled_thoughts, spilled_poetry, tumblr_poem, queer_poetry, indigenous_poetry, spilled_ink, spilled_thoughts, spilled_words, tumblr, tumblr_poem, queer_poetry, tumblr_poem, indigenous_poetry, spilled_ink, spilled_thoughts, queer_poetry, indigenous_poetry, spilled_ink, spilled_thoughts, spilled_poetry, tumblr_poetry, queer_poetry, indigenous_poetry, spilled_thoughts, spilled_words, tumblr_poetry, queer_poetry, indigenous_poetry, spilled_ink, tumblr_poetry, queer_poetry, indigenous_poetry, spilled_ink, spilled_words, spilled_poetry, tumblr_poetry, indigenous_poetry, spilled_words, spilled_ink, tumblr_poem, queer_poetry, indigenous_poetry, spilled_ink, spilled_words, tumblr_poetry, queer_poetry, indigenous_poetry, sun, spilled_ink, spilled_words, tumblr_poem, queer_poetry, indigenous_poetry, spilled_ink, spilled_thoughts, tumblr_poem, queer_poetry, indigenous_poetry, spilled_word, spilled_ink, tumblr_poem, indigenous_poetry, spilled_ink, spilled_poetry, tumblr_poem, spilled_ink, spilled_thoughts, untitled, indigenous_poetry, queer_poetry, tumblr_poem, queer_poetry, queerpoetry, lgbtpoetry, native_american_poetry, indigenous_poetry, queer_indigenous_poetry, tumblr_poem, indigenous_poetry, queer_poetry, spilled_ink, spilled_poetry, tumblr_poem, sun, dawn, void, queer_poetry, spilled_ink, spilled_poetry, tumblr_poem, indigenous_poetry, queer_poetry, ...]"
1371586,"[today, day3_schedule, day3_schedule, day_four, results, day_four, schedule, day_four, schedule, day_four, results, day_four, schedule, estonia, luxembourg, men, day_four, schedule, final, slovakia, poland, men, day3_results, day3_results, day3_results, day3_results, day3_results, results, day3_results, day_3_schedule, day3_schedule, day3_results, day_four, schedule, day_three, jersey, poland, men, results, day_three, results, men, croatia, slovakia, day_four, schedule, croatia, russia, day_four, greece, isle_of_man, men, schedule, day_four, romania, guernsey, men, schedule, day_four, results, day_three, results, men, croatia, slovakia, day_three, luxembourg, women, results, croatia, results, day_four, schedule, day_four, results, women, malta, jersey, day_four, men, results, jersey, croatia, day_four, men, results, russia, lithuania, day_four, men, results, slovakia, poland, results, finals, results, finals, malta, women, day3_schedule, ...]"
1963332,"[txt, txt, same, txt, jfc, txt, purple, \o/, txt, star_wars, :/, txt, txt, crack, omg, true, txt, txt, iasip, txt, film, txt, lmao, mcu, true, same, glenn_howerton, honestly, beauty_and_the_beast, film, iasip, txt, lmao, glenn_howerton, iasip, txt, fuck, i_can't, true_tho, :/, iasip, glenn_howerton, txt, chaotic_neutral, intp, slytherin, txt, tbd, black_hole, star_wars, rebels, :/, txt, iasip, txt, txt, same, music, txt, thor, marvel, lol, true, txt, yassss, bring_it, skam, txt, crack, txt, txt, wank, preach, txt, txt, crack, txt, bookmark, text, wine, txt, true, iasip_cast, txt, iasip, it's_always_sunny_in_philadelphia, mac, txt, txt, iasip, iasip, txt, crack,

In [65]:
agg_tags.rename(columns={'parsed_tags_minfreq3': 'agg_parsed_tags_minfreq3'}, inplace=True)
agg_tags.columns

Index(['agg_parsed_tags_minfreq3'], dtype='object')

In [68]:
descs_tags = pd.merge(train_descs, agg_tags, left_on=['tumblog_id'], right_index=True)
len(descs_tags)

16214

In [71]:
top = {}
for t in [100, 500, 1000, 10000]:
    top[t] = [wd for wd,_ in tag_ctr.most_common(t)]
top['all'] = list(tag_ctr.keys())

In [89]:
category_tag_ctr = {}
mi = {}
n_items = 'all'
# for tag, category in tqdm(itertools.product(top[n_items], categories), total=n_items*len(categories)):
# for c in categories:
# for c in ['gender/sexuality']:
categories = ['weight', 'roleplay']
for c in categories:
    print(c)
    category_tag_ctr[c] = {}
    cat_true = descs_tags[descs_tags[c]==True]
    for tag in tqdm(top[n_items]):
        # Count a joint occurrence of (category, tag) as a description having at least one occurrence of a tag
        category_tag_ctr[c][tag] = sum([tag in t for t in cat_true['agg_parsed_tags_minfreq3']])
#         mask = [tag in t[0] and t[1] for t in zip(descs_tags['parsed_tags_minfreq3'], descs_tags[category])]
#         category_tag_ctr[(tag,category)] = sum(mask)

    len(category_tag_ctr)

    mi[c] = {}
    for t, count in category_tag_ctr[c].items():
        mi[c][t] = count/(category_ctr[c] * tag_ctr[t])
len(mi)

weight



roleplay


2

In [91]:
[t for t,_ in sorted(mi['roleplay'].items(), key=itemgetter(1), reverse=True) if tag_ctr[t] > 1]

['creatingxdragons',
 'theinstituteclosed',
 'meinior',
 'heartlessvoice',
 'i_hope_this_is_okay_!!!',
 'specify_a_muse',
 "it's_totally_fine!",
 'so_sorry_guys',
 '*claps_hands*',
 "hope_you_don't_mind",
 'pleasepleaseplease',
 'unbreakablehonor',
 'lucentiox',
 'singlesighted',
 'themagicaluniverse',
 'sociantas',
 'mobile_lol',
 'thevioletandroid',
 'rfallsooc',
 'herxpearl',
 '(visage.)',
 '//:)',
 'and_happy',
 'ccandidum',
 'omg_xd',
 'bestpoeldancerofthegalaxy',
 'preciousyellowidiot',
 'dash_commentary;',
 'ic_for_ooc',
 'jerkisms',
 'meanxgreenxteam',
 'because_why_not?',
 'potiustia',
 'kingslight',
 'fearlesshonor',
 'desircble',
 'branchchieffaba',
 "i'm???",
 'this_should_be_interesting',
 'y_a_s',
 'the_mun',
 '.dash_games.',
 'fandomless_oc_rp',
 'bless_her',
 'princesshcrxine',
 'crack?',
 'food_mention*',
 'allherechos',
 'small_rant',
 'coffincaught',
 'c_r_i_e_s',
 'ofrxvia',
 'accidentprcne',
 'loyalxservant',
 'rip_my_ass',
 'bypistol',
 'ultimatetalentizuru',
 'ju

In [90]:
[t for t,_ in sorted(mi['weight'].items(), key=itemgetter(1), reverse=True) if tag_ctr[t] > 1]

['pro-ana',
 '10_day_fast',
 'anatips',
 'skinny_girl',
 'send_me_meanspo',
 '24_hour_fast',
 'bodycheck',
 'day_14',
 'day_12',
 'personal_meanspo',
 'get_thin',
 'day_11',
 'day_15',
 'i_need_to_be_thin',
 "i'm_so_fat",
 'binged',
 'eds',
 'laxatives',
 'thinspo_challenge',
 'thinspo_tips',
 'tw_weight',
 'day_4',
 'thispiration',
 'measurements',
 'day_10',
 'underweight',
 'i_want_to_be_skinny',
 'trans_girls',
 'external',
 'ugw',
 'hipbones',
 '100_calories',
 'fasting_tips',
 'calorie_counting',
 'low_calorie',
 'planet',
 'body_shaming',
 'legspo',
 'kill_myself',
 'delicate',
 'weigtloss',
 'body_checks',
 'restriction',
 'thinsp',
 'ana_tip',
 'intermittentfasting',
 'lost_weight',
 'psych_ward',
 'ana_asks',
 'triggering',
 'starving_myself',
 "i'm_broken",
 'dying_to_be_thin',
 'goal_weight',
 'bingeing',
 'tricks',
 'starvation',
 'low_self_esteem',
 'salad',
 'hate_my_body',
 'day_1',
 'purge',
 'gw',
 'anorexya',
 'flat_stomach',
 'give_me_attention',
 'ana_buddy',
 'unw

In [84]:
[t for t,_ in sorted(mi['gender/sexuality'].items(), key=itemgetter(1), reverse=True) if tag_ctr[t] > 1]

['gender_fluid',
 'i_literally_laughed_out_loud',
 'legitimate_question',
 'nbphobia',
 'i_hate_donald_trump',
 'that_would_be_nice',
 'theseus_scamander',
 'fantastic',
 "i've_been_thinking_about_this_all_day",
 "it's_whatever",
 'thank_goodness',
 'ignore_the_hand',
 'me_holy_shit',
 'alcohol_mention_tw',
 'sweetheart',
 'older_men',
 'not_again',
 'grchvms',
 'caps_in_tags_/',
 'safe_sex',
 'tw_ableism',
 'poor_babies',
 'is_my_aesthetic',
 'transman',
 '🔫',
 'sorry_if_this_is_annoying',
 'nsfw_mention_/',
 'other_half',
 'sasusaku',
 'naruhina',
 'but_god_damn',
 'vera_farmiga',
 'making_it_happen',
 'anti_antifa',
 'fidget_cube',
 'fjuck',
 'floriana_lima',
 'life_ref',
 'pleeeeease',
 'right_wing',
 'and_depressed',
 'scott_mccall',
 'um_yes???',
 "he's_so_pretty",
 'george_miller',
 'it_ruined_my_life',
 'hope_this_is_ok!!',
 'litten',
 'deleted',
 'misogyny_/',
 'transformation',
 'jeremy_allen_white',
 'exciting',
 'polyamorous',
 'please_share',
 'soothing',
 'good_boy',
 'th

## 1-time

In [ ]:
# Explore posts that contain certain tags
tag = 'older_men'
merged.loc[merged['parsed_tags_minfreq3'].map(lambda x: tag in x), ['tumblog_id', 'parsed_blog_description', 'body', 'parsed_tags_minfreq3']]

In [75]:
sorted(category_tag_ctr['gender/sexuality'].items(), key=itemgetter(1), reverse=True)

[('me', 605),
 ('personal', 320),
 ('nsfw', 276),
 ('yes', 219),
 ('same', 209),
 ('lmao', 197),
 ('lol', 197),
 ('important', 173),
 ('please', 168),
 ('omg', 157),
 ('ooc', 149),
 ('fuck', 149),
 ('submission', 134),
 ('about_me', 132),
 ('text', 130),
 ('long_post', 127),
 ('true', 114),
 ('!!!', 112),
 ('q', 105),
 ('ref', 104),
 ('oh_my_god', 100),
 ('mood', 97),
 ('pls', 96),
 ('writing', 95),
 ('<3', 93),
 ('fav', 93),
 ('idk', 93),
 ('depression', 92),
 ('honestly', 92),
 ('this', 92),
 ('ugh', 91),
 ('help', 89),
 ('fave', 88),
 ('thank_you', 82),
 ('lgbt', 82),
 ('signal_boost', 82),
 ('meme', 79),
 ('love', 79),
 ('god', 78),
 ('wow', 78),
 ('no', 77),
 ('yeah', 76),
 ('psa', 76),
 ('positivity', 72),
 ('star_wars', 70),
 ('mine', 68),
 (':)', 68),
 ('reference', 67),
 ('overwatch', 65),
 ('save', 65),
 ('tbh', 65),
 ('gay', 64),
 ('delete_later', 64),
 ('art', 64),
 ('bye', 63),
 ('replies', 62),
 (':(', 62),
 ('sorry', 62),
 ('nice', 61),
 ('holy_shit', 61),
 ('!!!!', 61),

In [45]:
pd.set_option('display.max_colwidth', -1)

In [47]:
# Explore blog descriptions that contain certain tags
tag = 'kikgirls'
merged.loc[merged['parsed_tags_minfreq3'].map(lambda x: tag in x), ['tumblog_id', 'parsed_blog_description',]]

tumblog_id  \
583980   331628644    
583982   331628644    
583984   331628644    
583986   331628644    
583987   331628644    
583988   331628644    
583990   331628644    
596756   331628644    
613753   331628644    
616993   331628644    
622458   331628644    
622476   331628644    
622485   331628644    
622499   331628644    
622507   331628644    
622557   331628644    
622580   331628644    
622601   331628644    
622618   331628644    
622640   331628644    
622691   331628644    
622692   331628644    
622722   331628644    
622740   331628644    
622745   331628644    
622754   331628644    
638269   331628644    
643247   331628644    
670301   331628644    
701108   331628644    
...            ...    
1455795  341022482    
1455796  341022482    
1473935  341022482    
1473946  341022482    
1473952  341022482    
1473954  341022482    
1473961  341022482    
1473967  341022482    
1473972  341022482    
1473977  341022482    
1473979  341022482    
1473994  341022482    
1474005  341022482    
1474011  341022482    
1474261  341022482    
1474279  341022482    
1480888  341022482    
1485379  341022482    
1485389  341022482    
1485460  341022482    
1490861  341022482    
1490866  341022482    
1490870  341022482    
1490954  341022482    
1490981  341022482    
1491299  341022482    
1512698  341022482    
1523608  341022482    
1523616  341022482    
1529271  341022482    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [36]:
tag = 'chat'
category = 'sexual orientation'
len(merged[merged[category]==True])
# mask = [tag in t[0] and t[1] for t in zip(merged['parsed_tags_minfreq3'], merged[category])]
# len(merged[mask])

130900